In [45]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from pathlib import Path
import time
import urllib.request
import urllib.error as uer

import lxml


import os
import csv

In [33]:
anime = []
for page in tqdm(range(0,400)):
    url = "https://myanimelist.net/topanime.php?limit " +str(page *50)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    for tag in soup.find_all("tr"):#cerco le righe della tabella
        links = tag.find_all("a")
        for link in links:
            if type(link.get("id")) == str and len(link.contents[0]) >1:
                anime.append((link.get("href")))
with open('urls.txt', 'w') as f:
    for item in tqdm(anime):
        f.write("%s\n" % item)      
            


100%|██████████| 20000/20000 [00:00<00:00, 682872.28it/s]


In [34]:
len(anime)

20000

In [22]:
#downloading the html pages
def html_downloading(l):
    n = []
    for i in range(3):
        try:
            file = '/Users/michele/ADM2021/article_ '+ str(i) + '.html'   
            urllib.request.urlretrieve(l[i], file)
        #if the link does not exist, we pass to the next link 
        except uer.HTTPError:
            pass
        except Exception as error:
            #if an exception occured, we append the number of the link to this list
            n.append(i)
            time.sleep(1200) #60s * 20m = 1200s
        time.sleep(2)
    
    #it will run only if some of the previous html was not downloaded because an exception occured 
    if n != []:
        for i in n:
            try:
                file = '/Users/michele/ADM2021/article_ '+ str(i) + '.html'
                urllib.request.urlretrieve(l[i], file)
            #if the link does not exist, we pass to the next link  
            except uer.HTTPError:
                pass
            except Exception as error:
                time.sleep(1200) #60s * 20m = 1200s
            time.sleep(2)


In [ ]:
html_downloading(anime)

In [52]:
def crawl_urls(n_pages, url_file):
    pathAncestor = os.path.join("./", "htmlpages") # Path
    os.mkdir(pathAncestor) # create the folder in the path
    
    for i in range(1,n_pages+1):
        os.makedirs(os.path.join(pathAncestor, 'page ' + str(i))) # create sequentially the folders interested
    
    UrlsFiles = open(url_file, "r") # open the file in "read (r)" mode

    headpart = "https://myanimelist.net"
    counter_pages = 0
    counter_html = 0
    for x in UrlsFiles: # crawl each Urls associated to the book to be sure to download the corresponding html article
        if counter_html % 100 == 0: # check every hundred html page to change the folder where we insert the html article
            counter_pages = counter_pages + 1 

        counter_html = counter_html + 1

        subdirectory = pathAncestor + "/page " + str(counter_pages) # select the corresponding folder to insert the html article
        article_name = "/article_"+str(counter_html)+".html" # set the number of i-th book

        complete_path = subdirectory + article_name # insert the new complete path where create the html file
        with open(complete_path, "wb") as ip_file:
            link = headpart + x
            import time

            page = requests.get(link)
            try:
                page = requests.get(link)
                break
            except:
                print("Connection refused by the server..")

                time.sleep(5)
                print("Was a nice sleep, now let me continue...")
                continue
             # request the page


            soup = BeautifulSoup(page.text, features='lxml')

            ip_file.write(soup.encode('utf-8'))
            ip_file.close()

    UrlsFiles.close()


In [ ]:
for x in urls.text:
    request.get(url)